In [36]:
import xml.dom.minidom
domobj = xml.dom.minidom.parse('map.xml')
elementobj=domobj.documentElement
nodeobj=elementobj.getElementsByTagName('node')
wayobj=elementobj.getElementsByTagName('way')
node_dict={}
for i in range(len(nodeobj)):
    node_dict[nodeobj[i].getAttribute('id')]=(nodeobj[i].getAttribute('lon'),nodeobj[i].getAttribute('lat'))
def getValueByKey(obj,k):
    for tag in obj:
        if tag.getAttribute('k')==k:
            return tag.getAttribute('v')
    return ''
way_list=[]
for i in range(len(wayobj)):
    tagobj=wayobj[i].getElementsByTagName('tag')
    if getValueByKey(tagobj,'highway')=='primary':
        ndobj=wayobj[i].getElementsByTagName('nd')
        ndids=[nd.getAttribute('ref') for nd in ndobj]
        way_list.append((getValueByKey(tagobj,'name'),ndids))

In [7]:
import requests
import json
import threading
import queue
from fake_useragent import UserAgent
from progressbar import ProgressBar,Percentage,Bar,Timer,ETA,FileTransferSpeed
widgets = [
    'Progress: ',
    Percentage(), ' ',
    Bar('#'), ' ',
    Timer(), ' ',
    ETA(), ' ',
    FileTransferSpeed()
]
bar = ProgressBar(widgets=widgets, maxval=len(node_dict)).start()
q=queue.Queue()
thread_list=[]
def convert_task(ndids):
    gps_string=''
    for ndid in ndids:
        gps_string+=node_dict[ndid][0]+','+node_dict[ndid][1]+'|'
    server_host='https://restapi.amap.com/v3/assistant/coordinate/convert'
    parameters={
        'key':'053d8830d4ed8bfc7fac9a3a5d8b10cf',
        'locations':gps_string,
        'coordsys':'gps'
    }
    response=requests.get(url=server_host,
                 headers={'User-Agent':UserAgent(path='fakeuseragent.json').random},
                 params=parameters)
    response.encoding=response.apparent_encoding
    res=json.loads(response.text)
    coords=res['locations'].split(';')
    for i,ndid in enumerate(ndids):
        q.put((ndid,tuple(coords[i].split(','))))
node_dict_amap={}
for i in range(0,len(node_dict),40):
    iterate_nodes=list(node_dict.keys())[i:i+40]
    thread=threading.Thread(target=convert_task, args=(iterate_nodes,))
    thread_list.append(thread)
    thread.start()
    bar.update(i+1)
bar.finish()
for thread in thread_list:
    thread.join()
thread_list.clear()
while not q.empty():
    item=q.get()
    node_dict_amap[item[0]]=item[1]

Progress: 100% |##############| Elapsed Time: 0:12:03 Time: 0:12:03 936.04  B/s


In [8]:
import pickle
with open('map_nodes.pkl','wb') as f:
    pickle.dump(node_dict, f)
with open('map_nodes_amap.pkl','wb') as f:
    pickle.dump(node_dict_amap, f)
with open('map_ways.pkl','wb') as f:
    pickle.dump(way_list, f)

In [1]:
import pickle
node_dict={}
node_dict_amap={}
way_list=[]
with open('map_nodes.pkl','rb') as f:
    node_dict=pickle.load(f)
with open('map_nodes_amap.pkl','rb') as f:
    node_dict_amap=pickle.load(f)
with open('map_ways.pkl','rb') as f:
    way_list=pickle.load(f)

In [10]:
print(len(node_dict))
for way in way_list:
    if way[0]=='四平路':
        print([node_dict_amap[ref] for ref in way[1]])

677206
[('121.508343641494', '31.286169433594'), ('121.508276909723', '31.286039767796'), ('121.508210991754', '31.285907660591'), ('121.507644856771', '31.284783799914'), ('121.507049967448', '31.28364312066'), ('121.506650390625', '31.282882215712'), ('121.506416015625', '31.282436252171'), ('121.506234809028', '31.282092556424'), ('121.506128472223', '31.281898328994'), ('121.506041124132', '31.281730957032'), ('121.505888943143', '31.281546495226'), ('121.505843370226', '31.281483832466'), ('121.505806749132', '31.281425509983'), ('121.505730251737', '31.281271701389'), ('121.505616319445', '31.281034071181'), ('121.504907769098', '31.279642469619'), ('121.504751519098', '31.279347873264'), ('121.504514431424', '31.278879123264'), ('121.503926323785', '31.277751736112'), ('121.503887261285', '31.277689073351'), ('121.503781738282', '31.277510850695'), ('121.503465711806', '31.277073838976'), ('121.50328125', '31.27683186849'), ('121.502984483507', '31.276469726563'), ('121.50284776